# Pruning SSD Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained model already exists in the ```Models``` directory. Please refer to the notebook [Object Detection with SSD](SSD.ipynb) for more info about using a pretrained SSD model.

If you want to prune a Low-Rank model, you can use [this](SSD-Reduce.ipynb) notebook
to reduce the number of parameters in ```SSD```.

## Load and evaluate the original pretrained model

In [1]:
from fireball import Model, myPrint
from fireball.datasets.coco import CocoDSet
gpus = "upto4"

myPrint('\nPreparing Coco dataset ... ', False)
trainDs,testDs = CocoDSet.makeDatasets('Train,Test', batchSize=128, resolution=512, keepAr=False, numWorkers=4)
trainDs.batchSize = 64
myPrint('Done.')

orgFileName = "Models/SSD512RR.fbm"    # Reduced - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.initSession()
results = model.evaluate()


Preparing Coco dataset ... Done.

Reading from "Models/SSD512RR.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.
  Processed 5000 Sample. (Time: 56.36 Sec.)                              

Evaluating inference results for 5000 images ... 
  Calculating IoUs - Done (7.7 Seconds)                       
  Finding matches - Done (121.1 Seconds)                     
  Processing the matches - Done (4.1 Seconds)                    
Done (132.9 Seconds)

Average Precision (AP):
    IoU=0.50:0.95   Area: All      MaxDet: 100  = 0.248
    IoU=0.50        Area: All      MaxDet: 100  = 0.465
    IoU=0.75        Area: All      MaxDet: 100  = 0.240
    IoU=0.50:0.95   Area: Small    MaxDet: 100  = 0.095
    IoU=0.50:0.95   Area: Medium   MaxDet: 100  = 0.293
    IoU=0.50:0.95   Area: Large    MaxDet: 100  = 0.365
Average Recall (AR):
    IoU=0.50:0.95   Area: All      MaxDet: 1    = 0.230
    IoU=0.50:0.95   Area: All      MaxDet: 10   = 0.352
    IoU=0.50:0.95   Area: All      MaxDet:

## Pruning the model
Here we prune the model using the [pruneModel](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.pruneModel) class method of the model.

In [2]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.000005)


Reading model parameters from "Models/SSD512RR.fbm" ... Done.
Pruning 92 tensors using 36 workers ... 
   Pruning Parameters:
        mseUb ................ 0.000005
Pruning process complete (5.57 Sec.)
Now saving to "Models/SSD512RRP.fbm" ... Done.

Number of parameters: 23,147,860 -> 15,799,194 (7,348,666 pruned)
Model File Size: 92,598,151 -> 66,097,263 bytes


## Evaluate the pruned model
Compare the new number of parameters with the original. Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus=gpus)   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/SSD512RRP.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 512x512x3    512 512 3     None                      0          
S1_L1_CONV       512 512 3     KSP: 3 1 s               512 512 64    ReLU                      1,680      
S1_L2_CONV       512 512 64    KSP: 3 1 s               256 256 64    ReLU     MP(KSP):2 2 s    28,321     
S2_L1_CONV       256 256 64    KSP: 3 1 s               256 256 128   ReLU                      52,797     
S2_L2_CONV       256 256 128   KSP: 3 1 s               128 128 128   ReLU     MP(KSP):2 2 s    100,940    
S3_L1_CONV       128 128 128   KSP: 3 1 s, LR136        128 128 256   ReLU                      146,005    
S3_L2_CONV       128 128 256   KSP: 3 1 s

## Re-training after pruning
Here we first make a new model from the file created above. We then call the [train](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.train) method of the model to start the re-training.

After re-training, we run the [evaluate](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.evaluate) function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [4]:

model = Model.makeFromFile(prunedFileName, trainDs=trainDs, testDs=testDs,
                           numEpochs=5,
                           learningRate=(0.002, 0.0004),
                           optimizer="Momentum",
                           gpus=gpus)
model.initSession()
model.printLayersInfo()
model.printNetConfig()
model.train()
results = model.evaluate()

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)


Reading from "Models/SSD512RRP.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 512x512x3    512 512 3     None                      0          
S1_L1_CONV       512 512 3     KSP: 3 1 s               512 512 64    ReLU                      1,680      
S1_L2_CONV       512 512 64    KSP: 3 1 s               256 256 64    ReLU     MP(KSP):2 2 s    28,321     
S2_L1_CONV       256 256 64    KSP: 3 1 s               256 256 128   ReLU                      52,797     
S2_L2_CONV       256 256 128   KSP: 3 1 s               128 128 128   ReLU     MP(KSP):2 2 s    100,940    
S3_L1_CONV       128 128 128   KSP: 3 1 s, LR136        128 128 256   ReLU                      146,005    
S3_L2_CONV       128 128 256   KSP: 3 1 s

## Also look at

[Quantizing SSD Model](SSD-Quantize.ipynb)

[Exporting SSD Model to ONNX](SSD-ONNX.ipynb)

[Exporting SSD Model to TensorFlow](SSD-TF.ipynb)

[Exporting SSD Model to CoreML](SSD-CoreML.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with SSD](SSD.ipynb)

[Reducing number of parameters of SSD Model](SSD-Reduce.ipynb)